In [1]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa142.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "BkbVl5Bo8AolvJt"

In [3]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            if duration_unit == "d":
                barrier = "10000"
            else:
                barrier = "+0"
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "AUD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sendbar')
            ws.send(json.dumps(msg))
        else:
            print('here')
            amount = 60
            if 'error' in message.keys():
                barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                if message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    print('amount =',amount)
                    print(barrier)
                    print(contract_type)
                    print(duration)
                    print(duration_unit)
                    print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print('sent')
                ws.send(json.dumps(msg))   
            else:
                return_json.append(message)
                ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [4]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):
    
    spot = float(pd_series.Spot)
    
    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    digits = int( assets[assets['Asset']== symbol].digits )
    
    sigma = float( assets[assets['Asset']==symbol].IV )  
    
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1
    
    spot = round(spot/10**-n_digits)*10**-n_digits
    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400       
    
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if duration_unit != 'd':
        
        deltas = np.arange(1,10,2)/10 
    
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )
    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )
        
    else:
        
        temp_barriers = []
        
        displayed_barriers = []

        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )

        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10

        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 

        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)

        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)

        temp_barriers.append(max_strike)

        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)


    return displayed_barriers
        

In [5]:
def calc_ppp_ask (pd_series, t_step, s_step):

    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    stake = pd_series.Stake

    vol_charge = 1.025
 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400  
    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400

    r = pd_series.Discount_rate 
    
    contract_type = pd_series.Contract_type

    number_of_contracts = pd_series.API_PPP
        
    if s_step == 0:
        
        spot = float(pd_series.Spot)

        strike = pd_series.Barrier
    
    else:
        
        spot = float(tick_history(int(duration[0]), symbol, s_step))
        
        if duration_unit != 'd':
            
            strike = spot + float(pd_series.Displayed_barrier)
            
        else:
            
            strike = pd_series.Barrier
        
    sigma = float( assets[assets['Asset']==symbol].IV )
    
    if duration_unit != 'd':
        
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
        
    else:
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
        
    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))
    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))
    
    return stake / ( bs_price + delta_charge + bs_markup )

In [6]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [7]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [25]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    barrier = 0
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0" 
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "AUD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))
        else:
            amount = 60
            if 'error' in message.keys():
                if message['error']['details']['field'] == 'barrier':
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                elif message['error']['details']['field'] == 'amount':
                    amount = int(message['error']['details']['min_stake']) + 5
                    barrier = message['echo_req']['barrier']
                    if message['error']['details']['min_stake'] > 2000:
                        barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                        amount = 2

                print(amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print(msg)
                ws.send(json.dumps(msg))   

            else:

                return_json.append(message)
                ws.close()
            
    def on_close(ws, close_status_code, close_msg):
        print()             
 
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    barriers = return_json[-1]['proposal']['barrier_choices']
    
    if 'qa' in master_URL:
        
        return barriers, 0#return_json[-1]['proposal']['pricing_args']['discount_rate']
    
    else:
        
        return barriers, 'prod_api'

In [9]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []

    return_barrier = []
    
    return_discount_rate = []

    contract_ID = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        print('\n',message)
        barrier = 0
        
        amount = 60
        
        message = json.loads(message)
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    

        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                print('here')
                print(message['error']['message'])
                print(message['error']['message'].split()[-1][:-1])
                print(amount)
                amount = float(message['error']['message'].split()[-1][:-1]) 
                
                if 'ax' in message['error']['message']:
                    amount -= 5
                else: 
                    amount += 5
                    
                print('here',amount)
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)

            barrier = barriers[0][np.random.randint(len(barriers[0]))]

            return_barrier.append(barrier)

            return_discount_rate.append(barriers[1])
            
            print (symbol)
            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "AUD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }

            ws.send(json.dumps(msg))
            
        else:
            return_json.append(message)
            
            ws.close()
                                          
    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [36]:
def calc_ppp_bid (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['bid_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400  
        
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if t > 1 / 365:
        
        r = 0#proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    
    
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['current_spot'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))
        
        if t > 1 / 365:
        
            strike = float(proposal['proposal_open_contract']['barrier'])

        else:

            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

        
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)

    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))

    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts
    

In [11]:
def sell_contract(contract_id):
    
    contract_id = int(contract_id)
    
    temp_json = []
    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]

In [37]:
def calc_ppp_sold (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['sell_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  

    if t > 1 / 365:
        
        r =0#proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['exit_tick'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)


    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    bs_price =  (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price,price+bs_markup,bs_price)
    return price * number_of_contracts
    


In [13]:
def calc_ppp_exp (proposal, t_step, s_step):
    stake = float(proposal['proposal_open_contract']['sell_price'])
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    if s_step == 0:
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [16]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)
    print(len(barrier_df))
    if len(barrier_df) > 50 - 1:
        break
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        
        max_duration = 1440

    elif duration_unit == 'h':
        
        max_duration = 24

    elif duration_unit == 'd':
        
        max_duration = 365

    i = np.random.randint(5)
    
    if i < 2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1,max_duration)
        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)
    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]
    
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,1HZ50V,VANILLALONGCALL,9752.2100,1689909845,438m,"[1689909845, 1689936125]","[-178.6, -72.9, 1.0, 75.5, 184.0]",0,"{'echo_req': {'amount': 60, 'barrier': '+1.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 438, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ50V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['..."
1,R_50,VANILLALONGCALL,10129.4110,1689909846,119d,"[1689909847, 1700265599]","[7300.0, 7860.0, 8420.0, 8980.0, 9540.0, 10100.0, 11100.0, 12100.0, 13100.0, 14100.0, 15100.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10100.0000', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 119, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices':..."
2,R_100,VANILLALONGCALL,10098.5500,1689909848,90d,"[1689909849, 1697759999]","[6100.0, 6900.0, 7700.0, 8500.0, 9300.0, 10100.0, 12400.0, 14700.0, 17000.0, 19300.0, 21600.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10100.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 90, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': [..."
3,R_50,VANILLALONGCALL,10128.2397,1689909850,178d,"[1689909851, 1705363199]","[6900.0, 7540.0, 8180.0, 8820.0, 9460.0, 10100.0, 11440.0, 12780.0, 14120.0, 15460.0, 16800.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10100.0000', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 178, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices':..."
4,R_50,VANILLALONGPUT,10128.8285,1689909852,1440m,"[1689909853, 1689996253]","[-329.788, -134.255, 3.459, 143.077, 348.074]",0,"{'echo_req': {'amount': 60, 'barrier': '+3.4590', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'AUD', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['..."
5,1HZ10V,VANILLALONGCALL,10042.3900,1689909854,1043m,"[1689909855, 1689972435]","[-57.1, -23.3, 0.1, 23.6, 57.6]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.10', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 1043, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': [..."
6,R_50,VANILLALONGCALL,10125.7768,1689909856,1440m,"[1689909856, 1689996256]","[-329.788, -134.255, 3.459, 143.077, 348.074]",0,"{'echo_req': {'amount': 60, 'barrier': '+3.4590', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': [..."
7,1HZ10V,VANILLALONGCALL,10043.2800,1689909858,305d,"[1689909858, 1716335999]","[8970.0, 9180.0, 9390.0, 9600.0, 9810.0, 10040.0, 10300.0, 10560.0, 10820.0, 11080.0, 11330.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10040.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'AUD', 'duration': 305, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices':..."
8,1HZ75V,VANILLALONGPUT,10178.5800,1689909860,100d,"[1689909860, 1698623999]","[6700.0, 7400.0, 8100.0, 8800.0, 9500.0, 10200.0, 11800.0, 13400.0, 15000.0, 16600.0, 18200.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10200.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'curre

In [17]:
Calc_barrier_choices = [] 

for i in barrier_df.index:
    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )

barrier_df.Calc_barrier_choices = Calc_barrier_choices

print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]),\
      '\nMismatches listed below :')

barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]


50 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [28]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    clear_output(wait=True)
    print(len(PPP_df))
    
    if len(PPP_df)>15-1:
        break
        
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_50'
    duration_unit = random.choice(time_units)
#     duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
    
    if i<2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1, max_duration)
#     duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
                               

#     clear_output(wait=True)
clear_output(wait=True)

PPP_df


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,1HZ75V,VANILLALONGPUT,10190.4600,1689910246,60.0,17h,-169.70,10020.7600,"[1689910246, 1689971446]",439,0,0.88078,0,"{'echo_req': {'contract_id': 439, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10020.76', 'barrier_count': 1, 'bid_price': 54.54, 'buy_price': 60, 'contract_id': 439, 'contract_type': 'VANILLALONGPUT', 'currency'..."
1,R_75,VANILLALONGCALL,9991.0062,1689910250,60.0,1d,10140.0000,10140.0000,"[1689910251, 1690070399]",479,0,0.39112,0,"{'echo_req': {'contract_id': 479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10140.0000', 'barrier_count': 1, 'bid_price': 55.96, 'buy_price': 60, 'contract_id': 479, 'contract_type': 'VANILLALONGCALL', 'curren..."
2,R_75,VANILLALONGCALL,9989.3350,1689910256,26.0,1m,+5.3400,9994.6750,"[1689910256, 1689910316]",519,0,12.86992,0,"{'echo_req': {'contract_id': 519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9994.6750', 'barrier_count': 1, 'bid_price': 23.73, 'buy_price': 26, 'contract_id': 519, 'contract_type': 'VANILLALONGCALL', 'currenc..."
3,1HZ10V,VANILLALONGPUT,10040.2800,1689910260,60.0,1h,+13.80,10054.0800,"[1689910260, 1689913860]",559,0,4.18067,0,"{'echo_req': {'contract_id': 559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10054.08', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 559, 'contract_type': 'VANILLALONGPUT', 'currency'..."
4,1HZ100V,VANILLALONGPUT,9431.2800,1689910264,60.0,1m,-6.80,9424.4800,"[1689910264, 1689910324]",599,0,23.12138,0,"{'echo_req': {'contract_id': 599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9424.48', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': 60, 'contract_id': 599, 'contract_type': 'VANILLALONGPUT', 'currency':..."
5,1HZ50V,VANILLALONGCALL,9717.9100,1689910925,60.0,51d,9320.00,9320.0000,"[1689910925, 1694390399]",1099,0,0.06364,0,"{'echo_req': {'contract_id': 1099, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9320.00', 'barrier_count': 1, 'bid_price': 57.79, 'buy_price': 60, 'contract_id': 1099, 'contract_type': 'VANILLALONGCALL', 'currenc..."
6,1HZ25V,VANILLALONGCALL,10173.7400,1689910929,60.0,24h,+71.10,10244.8400,"[1689910930, 1689997330]",1119,0,2.28300,0,"{'echo_req': {'contract_id': 1119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10244.84', 'barrier_count': 1, 'bid_price': 55.02, 'buy_price': 60, 'contract_id': 1119, 'contract_type': 'VANILLALONGCALL', 'curren..."
7,1HZ75V,VANILLALONGCALL,10187.7100,1689910933,60.0,365d,20240.00,20240.0000,"[1689910934, 1721519999]",1139,0,0.05826,0,"{'echo_req': {'contract_id': 1139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '20240.00', 'barrier_count': 1, 'bid_price': 52.63, 'buy_price': 60, 'contract_id': 1139, 'contract_type': 'VANILLALONGCALL', 'curren..."
8,R_10,VANILLALONGCALL,10033.5220,1689910936,60.0,341d,11400.000,11400.0000,"[1689910938, 1719446399]",1159,0,1.26170,0,"{'echo_req': {'contract_id': 1159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '11400.000', 'barrier_count': 1, 'bid_price': 49.53, 'buy_price': 60, 'contract_id': 1159, 'contract_type': 'VANILLALONGCALL', 'curre..."
9,1HZ100V,VANILLALONGCALL,9355.5300,1689910984,60.0,1d,8980.00,8980.0000,"[1689910984, 1690070399]",

#### Mismatches below are due assumming discount rate = 0 in calculations here

In [29]:
Calc_ppp = [] 

for i in PPP_df.index:
    
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )

PPP_df.Calc_PPP = Calc_ppp

clear_output(wait = True)

print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]),\
      '\nMismatches listed below :')

df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]

df

15 Test cases 
Mismatches = 7 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
1,R_75,VANILLALONGCALL,9991.0062,1689910250,60.0,1d,10140.0000,10140.0,"[1689910251, 1690070399]",479,0,0.39112,0.391040,"{'echo_req': {'contract_id': 479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10140.0000', 'barrier_count': 1, 'bid_price': 55.96, 'buy_price': 60, 'contract_id': 479, 'contract_type': 'VANILLALONGCALL', 'curren..."
5,1HZ50V,VANILLALONGCALL,9717.9100,1689910925,60.0,51d,9320.00,9320.0,"[1689910925, 1694390399]",1099,0,0.06364,0.063267,"{'echo_req': {'contract_id': 1099, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9320.00', 'barrier_count': 1, 'bid_price': 57.79, 'buy_price': 60, 'contract_id': 1099, 'contract_type': 'VANILLALONGCALL', 'currenc..."
7,1HZ75V,VANILLALONGCALL,10187.7100,1689910933,60.0,365d,20240.00,20240.0,"[1689910934, 1721519999]",1139,0,0.05826,0.055724,"{'echo_req': {'contract_id': 1139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '20240.00', 'barrier_count': 1, 'bid_price': 52.63, 'buy_price': 60, 'contract_id': 1139, 'contract_type': 'VANILLALONGCALL', 'curren..."
8,R_10,VANILLALONGCALL,10033.5220,1689910936,60.0,341d,11400.000,11400.0,"[1689910938, 1719446399]",1159,0,1.26170,1.209902,"{'echo_req': {'contract_id': 1159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '11400.000', 'barrier_count': 1, 'bid_price': 49.53, 'buy_price': 60, 'contract_id': 1159, 'contract_type': 'VANILLALONGCALL', 'curre..."
9,1HZ100V,VANILLALONGCALL,9355.5300,1689910984,60.0,1d,8980.00,8980.0,"[1689910984, 1690070399]",1279,0,0.12118,0.121151,"{'echo_req': {'contract_id': 1279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '8980.00', 'barrier_count': 1, 'bid_price': 58.67, 'buy_price': 60, 'contract_id': 1279, 'contract_type': 'VANILLALONGCALL', 'currenc..."
11,R_25,VANILLALONGCALL,9986.7260,1689910992,60.0,1d,9960.000,9960.0,"[1689910992, 1690070399]",1359,0,0.69272,0.692579,"{'echo_req': {'contract_id': 1359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9960.000', 'barrier_count': 1, 'bid_price': 57.58, 'buy_price': 60, 'contract_id': 1359, 'contract_type': 'VANILLALONGCALL', 'curren..."
13,1HZ75V,VANILLALONGCALL,10178.5000,1689911000,60.0,165d,17340.00,17340.0,"[1689911000, 1704239999]",1439,0,0.11518,0.112781,"{'echo_req': {'contract_id': 1439, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '17340.00', 'barrier_count': 1, 'bid_price': 51.6, 'buy_price': 60, 'contract_id': 1439, 'contract_type': 'VANILLALONGCALL', 'currenc..."


In [34]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    
    print(len(Bid_df))
    
    if len(Bid_df)>20-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_10'
    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               

    clear_output(wait=True)
Bid_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ75V,VANILLALONGCALL,10171.5900,1689911128,60.0,270d,8320.00,8320.0000,"[1689911128, 1713311999]",1519,0,0.01810,0,0,0,0,"{'echo_req': {'contract_id': 1519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '8320.00', 'barrier_count': 1, 'bid_price': 58.13, 'buy_price': 60, 'contract_id': 1519, 'contract_type': 'VANILLALONGCALL', 'currenc..."
1,R_10,VANILLALONGPUT,10036.4100,1689911132,60.0,196d,11060.000,11060.0000,"[1689911132, 1706918399]",1539,0,0.05802,0,0,0,0,"{'echo_req': {'contract_id': 1539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '11060.000', 'barrier_count': 1, 'bid_price': 59.63, 'buy_price': 60, 'contract_id': 1539, 'contract_type': 'VANILLALONGPUT', 'curren..."
2,1HZ50V,VANILLALONGCALL,9715.8500,1689911136,60.0,204d,9060.00,9060.0000,"[1689911136, 1707609599]",1559,0,0.03459,0,0,0,0,"{'echo_req': {'contract_id': 1559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9060.00', 'barrier_count': 1, 'bid_price': 57.71, 'buy_price': 60, 'contract_id': 1559, 'contract_type': 'VANILLALONGCALL', 'currenc..."
3,R_10,VANILLALONGCALL,10036.4440,1689911140,60.0,827m,+20.990,10057.4340,"[1689911140, 1689960760]",1579,0,7.58849,0,0,0,0,"{'echo_req': {'contract_id': 1579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10057.434', 'barrier_count': 1, 'bid_price': 54.75, 'buy_price': 60, 'contract_id': 1579, 'contract_type': 'VANILLALONGCALL', 'curre..."
4,1HZ10V,VANILLALONGPUT,10041.1200,1689911144,60.0,14h,+51.70,10092.8200,"[1689911144, 1689961544]",1599,0,1.11581,0,0,0,0,"{'echo_req': {'contract_id': 1599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10092.82', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 1599, 'contract_type': 'VANILLALONGPUT', 'currenc..."
5,R_10,VANILLALONGCALL,10036.9490,1689911148,60.0,1428m,-27.270,10009.6790,"[1689911148, 1689996828]",1619,0,1.59107,0,0,0,0,"{'echo_req': {'contract_id': 1619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10009.679', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': 60, 'contract_id': 1619, 'contract_type': 'VANILLALONGCALL', 'curre..."
6,1HZ25V,VANILLALONGPUT,10176.5500,1689911151,60.0,1015m,+59.60,10236.1500,"[1689911152, 1689972052]",1659,0,0.73456,0,0,0,0,"{'echo_req': {'contract_id': 1659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10236.15', 'barrier_count': 1, 'bid_price': 58.64, 'buy_price': 60, 'contract_id': 1659, 'contract_type': 'VANILLALONGPUT', 'currenc..."
7,1HZ75V,VANILLALONGCALL,10176.6100,1689911156,60.0,190d,15560.00,15560.0000,"[1689911156, 1706399999]",1699,0,0.07020,0,0,0,0,"{'echo_req': {'contract_id': 1699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '15560.00', 'barrier_count': 1, 'bid_price': 53.39, 'buy_price': 60, 'contract_id': 1699, 'contract_type': 'VANILLALONGCALL', 'curren..."
8,1HZ75V,VANILLALONGCALL,10173.7400,1689911159,60.0,889m,+171.30,10345.0400,"[1689911160, 1689964500]",1739,0,0.97749,0,0,0,0,"{'echo_req': {'contract_id': 1739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10345.04', 'barrier_count': 1, 'bid_price': 55.25, 'buy_price': 60, 'contract_id': 1739, 'contract_type': 'VANILLALONGCALL', 'curren..."
9,1

In [40]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in Bid_df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )       
        bid_prop.append(proposal)   
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[round(Bid_df.API_bid,8) - np.round(Bid_df.Calc_bid,2) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[round(Bid_df.API_bid,8) - np.round(Bid_df.Calc_bid,2) != 0]
df

20 Test cases 
Mismatches = 10 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ75V,VANILLALONGCALL,10171.5900,1689911128,60.0,270d,8320.00,8320.000,"[1689911128, 1713311999]",1519,0,0.01810,1689916382,57.03,59.000136,"{'echo_req': {'contract_id': 1519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '8320.00', 'barrier_count': 1, 'bid_price': 57.03, 'buy_price': 60, 'contract_id': 1519, 'contract_type': 'VANILLALONGCALL', 'currenc...","{'echo_req': {'contract_id': 1519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '8320.00', 'barrier_count': 1, 'bid_price': 58.13, 'buy_price': 60, 'contract_id': 1519, 'contract_type': 'VANILLALONGCALL', 'currenc..."
1,R_10,VANILLALONGPUT,10036.4100,1689911132,60.0,196d,11060.000,11060.000,"[1689911132, 1706918399]",1539,0,0.05802,1689916384,60.48,62.021250,"{'echo_req': {'contract_id': 1539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '11060.000', 'barrier_count': 1, 'bid_price': 60.48, 'buy_price': 60, 'contract_id': 1539, 'contract_type': 'VANILLALONGPUT', 'curren...","{'echo_req': {'contract_id': 1539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '11060.000', 'barrier_count': 1, 'bid_price': 59.63, 'buy_price': 60, 'contract_id': 1539, 'contract_type': 'VANILLALONGPUT', 'curren..."
2,1HZ50V,VANILLALONGCALL,9715.8500,1689911136,60.0,204d,9060.00,9060.000,"[1689911136, 1707609599]",1559,0,0.03459,1689916385,60.94,62.553637,"{'echo_req': {'contract_id': 1559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9060.00', 'barrier_count': 1, 'bid_price': 60.94, 'buy_price': 60, 'contract_id': 1559, 'contract_type': 'VANILLALONGCALL', 'currenc...","{'echo_req': {'contract_id': 1559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9060.00', 'barrier_count': 1, 'bid_price': 57.71, 'buy_price': 60, 'contract_id': 1559, 'contract_type': 'VANILLALONGCALL', 'currenc..."
3,R_10,VANILLALONGCALL,10036.4440,1689911140,60.0,827m,+20.990,10057.434,"[1689911140, 1689960760]",1579,0,7.58849,1689916386,22.02,22.025530,"{'echo_req': {'contract_id': 1579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10057.434', 'barrier_count': 1, 'bid_price': 22.02, 'buy_price': 60, 'contract_id': 1579, 'contract_type': 'VANILLALONGCALL', 'curre...","{'echo_req': {'contract_id': 1579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10057.434', 'barrier_count': 1, 'bid_price': 54.75, 'buy_price': 60, 'contract_id': 1579, 'contract_type': 'VANILLALONGCALL', 'curre..."
7,1HZ75V,VANILLALONGCALL,10176.6100,1689911156,60.0,190d,15560.00,15560.000,"[1689911156, 1706399999]",1699,0,0.07020,1689916394,51.51,52.783685,"{'echo_req': {'contract_id': 1699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '15560.00', 'barrier_count': 1, 'bid_price': 51.51, 'buy_price': 60, 'contract_id': 1699, 'contract_type': 'VANILLALONGCALL', 'curren...","{'echo_req': {'contract_id': 1699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '15560.00', 'barrier_count': 1, 'bid_price': 53.39, 'buy_price': 60, 'contract_id': 1699, 'contract_type': 'VANILLALONGCALL', 'curren..."
10,R_50,VANILLALONGCALL,10119.0507,1689911280,60.0,352d,6100.0000,6100.000,"[1689911281, 1720396799]",2039,0,0

In [56]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
while True:
    
    print(len(Sold_df))
    
    if len(Sold_df) > 10 - 1:
        break
    
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    asset = 'R_75'
    duration_unit = random.choice(['m','h','d'])
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)        
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), proposal]
                               

    clear_output(wait=True)
    
Sold_df

10


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json
0,R_75,VANILLALONGCALL,10145.7730,1689916582,60.0,823m,+162.9360,10308.7090,"[1689916583, 1689965963]",2779,0,1.01208,"{'echo_req': {'contract_id': 2779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10308.7090', 'barrier_count': 1, 'bid_price': 54.69, 'buy_price': 60, 'contract_id': 2779, 'contract_type': 'VANILLALONGCALL', 'curr..."
1,R_75,VANILLALONGCALL,10143.0869,1689916586,60.0,1370m,+212.4120,10355.4989,"[1689916587, 1689998787]",2819,0,0.78815,"{'echo_req': {'contract_id': 2819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10355.4989', 'barrier_count': 1, 'bid_price': 54.67, 'buy_price': 60, 'contract_id': 2819, 'contract_type': 'VANILLALONGCALL', 'curr..."
2,R_75,VANILLALONGCALL,10144.4720,1689916590,60.0,13h,+158.4720,10302.9440,"[1689916591, 1689963391]",2859,0,1.03944,"{'echo_req': {'contract_id': 2859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10302.9440', 'barrier_count': 1, 'bid_price': 54.7, 'buy_price': 60, 'contract_id': 2859, 'contract_type': 'VANILLALONGCALL', 'curre..."
3,R_75,VANILLALONGPUT,10142.8601,1689916594,60.0,954m,-400.2820,9742.5781,"[1689916595, 1689973835]",2899,0,3.49040,"{'echo_req': {'contract_id': 2899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9742.5781', 'barrier_count': 1, 'bid_price': 50.01, 'buy_price': 60, 'contract_id': 2899, 'contract_type': 'VANILLALONGPUT', 'curren..."
4,R_75,VANILLALONGCALL,10139.5196,1689916598,60.0,315d,31400.0000,31400.0000,"[1689916599, 1717199999]",2939,0,0.21078,"{'echo_req': {'contract_id': 2939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '31400.0000', 'barrier_count': 1, 'bid_price': 47.26, 'buy_price': 60, 'contract_id': 2939, 'contract_type': 'VANILLALONGCALL', 'curr..."
5,R_75,VANILLALONGCALL,10143.1963,1689916602,60.0,760m,+156.3570,10299.5533,"[1689916603, 1689962203]",2979,0,1.05307,"{'echo_req': {'contract_id': 2979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10299.5533', 'barrier_count': 1, 'bid_price': 54.7, 'buy_price': 60, 'contract_id': 2979, 'contract_type': 'VANILLALONGCALL', 'curre..."
6,R_75,VANILLALONGCALL,10137.6048,1689916606,60.0,348d,24320.0000,24320.0000,"[1689916607, 1720051199]",3019,0,0.09221,"{'echo_req': {'contract_id': 3019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '24320.0000', 'barrier_count': 1, 'bid_price': 50.7, 'buy_price': 60, 'contract_id': 3019, 'contract_type': 'VANILLALONGCALL', 'curre..."
7,R_75,VANILLALONGPUT,10139.2384,1689916610,60.0,20h,-447.1680,9692.0704,"[1689916611, 1689988611]",3079,0,3.11149,"{'echo_req': {'contract_id': 3079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9692.0704', 'barrier_count': 1, 'bid_price': 50.02, 'buy_price': 60, 'contract_id': 3079, 'contract_type': 'VANILLALONGPUT', 'curren..."
8,R_75,VANILLALONGPUT,10134.7415,1689916614,60.0,1002m,+438.5310,10573.2725,"[1689916615, 1689976735]",3119,0,0.13167,"{'echo_req': {'contract_id': 3119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10573.2725', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3119, 'contract_type': 'VANILLALONGPUT', 'curre..."
9,R_75,VANILLALONGCALL,10163.5025,1689917624,60.0,964m,+5.2630,10168.7655,"[1689917624, 1689975464]",46

In [57]:
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]
df

10 Test cases 
Mismatches = 4 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json,Sold_time,API_sold,Calc_sold,Sold_prop
3,R_75,VANILLALONGPUT,10142.8601,1689916594,60.0,954m,-400.2820,9742.5781,"[1689916595, 1689973835]",2899,0,3.49040,"{'echo_req': {'contract_id': 2899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9742.5781', 'barrier_count': 1, 'bid_price': 50.01, 'buy_price': 60, 'contract_id': 2899, 'contract_type': 'VANILLALONGPUT', 'curren...",1689917642,43.19,43.195299,"{'echo_req': {'contract_id': 2899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'audit_details': {'contract_start': [{'epoch': 1689916590, 'tick': 10144.472, 'tick_display_value': '10144.4720'}, {'epoch': 1689916592, 'tick':..."
4,R_75,VANILLALONGCALL,10139.5196,1689916598,60.0,315d,31400.0000,31400.0000,"[1689916599, 1717199999]",2939,0,0.21078,"{'echo_req': {'contract_id': 2939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '31400.0000', 'barrier_count': 1, 'bid_price': 47.26, 'buy_price': 60, 'contract_id': 2939, 'contract_type': 'VANILLALONGCALL', 'curr...",1689917646,47.53,49.422294,"{'echo_req': {'contract_id': 2939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'audit_details': {'contract_start': [{'epoch': 1689916594, 'tick': 10142.8601, 'tick_display_value': '10142.8601'}, {'epoch': 1689916596, 'tick'..."
6,R_75,VANILLALONGCALL,10137.6048,1689916606,60.0,348d,24320.0000,24320.0000,"[1689916607, 1720051199]",3019,0,0.09221,"{'echo_req': {'contract_id': 3019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '24320.0000', 'barrier_count': 1, 'bid_price': 50.7, 'buy_price': 60, 'contract_id': 3019, 'contract_type': 'VANILLALONGCALL', 'curre...",1689917654,50.94,53.162088,"{'echo_req': {'contract_id': 3019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'audit_details': {'contract_start': [{'epoch': 1689916602, 'tick': 10143.1963, 'tick_display_value': '10143.1963'}, {'epoch': 1689916604, 'tick'..."
7,R_75,VANILLALONGPUT,10139.2384,1689916610,60.0,20h,-447.1680,9692.0704,"[1689916611, 1689988611]",3079,0,3.11149,"{'echo_req': {'contract_id': 3079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9692.0704', 'barrier_count': 1, 'bid_price': 50.02, 'buy_price': 60, 'contract_id': 3079, 'contract_type': 'VANILLALONGPUT', 'curren...",1689917656,45.24,45.247170,"{'echo_req': {'contract_id': 3079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'audit_details': {'contract_start': [{'epoch': 1689916606, 'tick': 10137.6048, 'tick_display_value': '10137.6048'}, {'epoch': 1689916608, 'tick'..."


In [49]:
# Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
while True:
    
    print(len(Expired_df))
    
    if len(Expired_df)> 20-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(1, 45)
#     duration=1
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]
                               

    clear_output(wait=True)
Expired_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,1HZ10V,VANILLALONGPUT,10043.350,1689917117,60.0,16m,+2.90,10046.250,"[1689917117, 1689918077]",3399,0,14.98012,0,0,"{'echo_req': {'contract_id': 3399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10046.25', 'barrier_count': 1, 'bid_price': 58.54, 'buy_price': 60, 'contract_id': 3399, 'contract_type': 'VANILLALONGPUT', 'currenc..."
1,1HZ100V,VANILLALONGPUT,9558.100,1689917121,60.0,40m,-43.50,9514.600,"[1689917121, 1689919521]",3419,0,3.61414,0,0,"{'echo_req': {'contract_id': 3419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9514.60', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': 60, 'contract_id': 3419, 'contract_type': 'VANILLALONGPUT', 'currency..."
2,1HZ10V,VANILLALONGCALL,10042.800,1689917267,60.0,10m,-5.60,10037.200,"[1689917267, 1689917867]",3959,0,10.29471,0,0,"{'echo_req': {'contract_id': 3959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10037.20', 'barrier_count': 1, 'bid_price': 59.6, 'buy_price': 60, 'contract_id': 3959, 'contract_type': 'VANILLALONGCALL', 'currenc..."
3,1HZ75V,VANILLALONGPUT,10069.870,1689917271,60.0,25m,+67.30,10137.170,"[1689917271, 1689918771]",3999,0,0.85746,0,0,"{'echo_req': {'contract_id': 3999, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10137.17', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3999, 'contract_type': 'VANILLALONGPUT', 'currenc..."
4,R_25,VANILLALONGCALL,9986.142,1689917274,60.0,30m,+0.020,9986.162,"[1689917275, 1689919075]",4039,0,7.78950,0,0,"{'echo_req': {'contract_id': 4039, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9986.162', 'barrier_count': 1, 'bid_price': 57.07, 'buy_price': 60, 'contract_id': 4039, 'contract_type': 'VANILLALONGCALL', 'curren..."
5,1HZ75V,VANILLALONGPUT,10067.430,1689917279,60.0,19m,-58.10,10009.330,"[1689917279, 1689918419]",4079,0,25.68204,0,0,"{'echo_req': {'contract_id': 4079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10009.33', 'barrier_count': 1, 'bid_price': 49.83, 'buy_price': 60, 'contract_id': 4079, 'contract_type': 'VANILLALONGPUT', 'currenc..."
6,R_100,VANILLALONGCALL,10092.450,1689917320,60.0,25m,+89.90,10182.350,"[1689917321, 1689918821]",4219,0,16.70742,0,0,"{'echo_req': {'contract_id': 4219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10182.35', 'barrier_count': 1, 'bid_price': 49.75, 'buy_price': 60, 'contract_id': 4219, 'contract_type': 'VANILLALONGCALL', 'curren..."
7,R_50,VANILLALONGPUT,10242.727,1689917324,60.0,41m,-57.4640,10185.263,"[1689917325, 1689919785]",4239,0,25.27836,0,0,"{'echo_req': {'contract_id': 4239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10185.2630', 'barrier_count': 1, 'bid_price': 49.9, 'buy_price': 60, 'contract_id': 4239, 'contract_type': 'VANILLALONGPUT', 'curren..."
8,R_25,VANILLALONGCALL,9988.814,1689917328,60.0,14m,-16.510,9972.304,"[1689917329, 1689918169]",4259,0,3.49313,0,0,"{'echo_req': {'contract_id': 4259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9972.304', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 4259, 'contract_type': 'VANILLALONGCALL', 'curren..."
9,1HZ100V,VANILLALONGCALL,9543.820,1689917332,60.0,9m,+0.10,9543.920,"[1689917333, 1689917873]

In [63]:
calc_exp = [] 

exp_time = []

api_exp = []

for i in Expired_df.index:
    
    if i >= len(exp_time):
        
        clear_output(wait = True)
                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)
        
        if proposal['proposal_open_contract']['is_sold'] == 1:
            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])

            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))

            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )
            
        else:
            
            exp_time.append(0)
            
            api_exp.append(0)

            calc_exp.append(0)
    
Expired_df['Exp_time'] = exp_time

Expired_df['API_exp'] = api_exp

Expired_df['Calc_exp'] = calc_exp

clear_output(wait = True)

print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]),\
      '\nMismatches listed below :')

df = Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]

df

20 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time


In [64]:
Expired_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
0,1HZ10V,VANILLALONGPUT,10043.350,1689917117,60.0,16m,+2.90,10046.250,"[1689917117, 1689918077]",3399,0,14.98012,0.00,0.000000,"{'echo_req': {'contract_id': 3399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10046.25', 'barrier_count': 1, 'bid_price': 58.54, 'buy_price': 60, 'contract_id': 3399, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689918077
1,1HZ100V,VANILLALONGPUT,9558.100,1689917121,60.0,40m,-43.50,9514.600,"[1689917121, 1689919521]",3419,0,3.61414,57.86,57.862381,"{'echo_req': {'contract_id': 3419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9514.60', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': 60, 'contract_id': 3419, 'contract_type': 'VANILLALONGPUT', 'currency...",1689919521
2,1HZ10V,VANILLALONGCALL,10042.800,1689917267,60.0,10m,-5.60,10037.200,"[1689917267, 1689917867]",3959,0,10.29471,64.86,64.856673,"{'echo_req': {'contract_id': 3959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10037.20', 'barrier_count': 1, 'bid_price': 59.6, 'buy_price': 60, 'contract_id': 3959, 'contract_type': 'VANILLALONGCALL', 'currenc...",1689917867
3,1HZ75V,VANILLALONGPUT,10069.870,1689917271,60.0,25m,+67.30,10137.170,"[1689917271, 1689918771]",3999,0,0.85746,0.00,0.000000,"{'echo_req': {'contract_id': 3999, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10137.17', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3999, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689918771
4,R_25,VANILLALONGCALL,9986.142,1689917274,60.0,30m,+0.020,9986.162,"[1689917275, 1689919075]",4039,0,7.78950,0.00,0.000000,"{'echo_req': {'contract_id': 4039, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9986.162', 'barrier_count': 1, 'bid_price': 57.07, 'buy_price': 60, 'contract_id': 4039, 'contract_type': 'VANILLALONGCALL', 'curren...",1689919075
5,1HZ75V,VANILLALONGPUT,10067.430,1689917279,60.0,19m,-58.10,10009.330,"[1689917279, 1689918419]",4079,0,25.68204,0.00,0.000000,"{'echo_req': {'contract_id': 4079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10009.33', 'barrier_count': 1, 'bid_price': 49.83, 'buy_price': 60, 'contract_id': 4079, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689918419
6,R_100,VANILLALONGCALL,10092.450,1689917320,60.0,25m,+89.90,10182.350,"[1689917321, 1689918821]",4219,0,16.70742,0.00,0.000000,"{'echo_req': {'contract_id': 4219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10182.35', 'barrier_count': 1, 'bid_price': 49.75, 'buy_price': 60, 'contract_id': 4219, 'contract_type': 'VANILLALONGCALL', 'curren...",1689918821
7,R_50,VANILLALONGPUT,10242.727,1689917324,60.0,41m,-57.4640,10185.263,"[1689917325, 1689919785]",4239,0,25.27836,448.11,448.109488,"{'echo_req': {'contract_id': 4239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '10185.2630', 'barrier_count': 1, 'bid_price': 49.9, 'buy_price': 60, 'contract_id': 4239, 'contract_type': 'VANILLALONGPUT', 'curren...",1689919785
8,R_25,VANILLALONGCALL,9988.814,1689917328,60.0,14m,-16.510,9972.304,"[1689917329, 1689918169]",4259,0,3.49313,0.00,0.000000,"{'echo_req': {'contract_id': 4259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200201, 'barrier': '9972.304', 'barrier_count': 1, 'bid_